# CLI Usage

While JSONLiteDB is *fundamentally* made to be a Python module, it comes with some minimal CLI tools to add, query, and dump the database. It does not currently support deleting rows, creating indexes, building advanced queries, etc. though those all may come in the future.

**Note**: This is a *shell* notebook

Clear a temp database so we are starting fresh

In [1]:
set +e
rm -rf my.db 1> /dev/null 2>&1

In [2]:
export JSONLITEDB_DISABLE_META=true # metadata in the file causes churn. Disable it for this demo
jsonlitedb --version

jsonlitedb-0.3.1


In [3]:
jsonlitedb insert my.db --stdin <<JSONL
    {"first": "John", "last": "Lennon", "born": 1940, "role": "guitar"}
    {"first": "Paul", "last": "McCartney", "born": 1942, "role": "bass"}
    {"first": "George", "last": "Harrison", "born": 1943, "role": "guitar"}
    {"first": "Ringo", "last": "Starr", "born": 1940, "role": "drums"}
JSONL

jsonlitedb query my.db --format count

4


Other ways to insert

In [4]:
jsonlitedb insert my.db --json ' {"first": "George", "last": "Martin", "born": 1926, "role": "producer"}'

jsonlitedb query my.db --format count

5


In [5]:
jsonlitedb query my.db first=George

{"first":"George","last":"Harrison","born":1943,"role":"guitar"}


{"first":"George","last":"Martin","born":1926,"role":"producer"}


You can also output in json instead of jsonlines. Note that it tries to still keep one item per line:

In [6]:
jsonlitedb query my.db first=George --format json --orderby born

[


{"first":"George","last":"Martin","born":1926,"role":"producer"},


{"first":"George","last":"Harrison","born":1943,"role":"guitar"}


]


More advanced ordering.

**Note**: You must either escape the negative (`--orderby "\-born"`) or use an `=` (`--orderby=-born`)

In [7]:
jsonlitedb query my.db --orderby first --orderby=-born

{"first":"George","last":"Harrison","born":1943,"role":"guitar"}


{"first":"George","last":"Martin","born":1926,"role":"producer"}


{"first":"John","last":"Lennon","born":1940,"role":"guitar"}


{"first":"Paul","last":"McCartney","born":1942,"role":"bass"}


{"first":"Ringo","last":"Starr","born":1940,"role":"drums"}


Add an index on first and last for faster queries. Make it unique then demonstrate how it works

In [8]:
jsonlitedb create-index my.db first last --unique

In [9]:
jsonlitedb insert my.db \
    --json '{"first":"Paul","last":"McCartney","role":"IMPOSTER"}' || echo "Command failed, continuing"

Error: UNIQUE constraint violation.


Index: ix_items_250e4243_UNIQUE


Hint: use --duplicates ignore or --duplicates replace.


SQLite: UNIQUE constraint failed: index 'ix_items_250e4243_UNIQUE'


Command failed, continuing


In [10]:
jsonlitedb insert my.db \
    --duplicates ignore \
    --json '{"first":"Paul","last":"McCartney","role":"IMPOSTER"}'

jsonlitedb query my.db first=Paul

{"first":"Paul","last":"McCartney","born":1942,"role":"bass"}


In [11]:
jsonlitedb insert my.db \
    --duplicates replace \
    --json '{"first":"Paul","last":"McCartney","role":"IMPOSTER"}'

jsonlitedb query my.db first=Paul

{"first":"Paul","last":"McCartney","role":"IMPOSTER"}


In [12]:
jsonlitedb delete my.db || echo "Command failed, continuing"

Error: refusing to delete all rows without filters.


Add one or more filters (e.g. name=Paul) or use --allow-empty.


Command failed, continuing


In [13]:
jsonlitedb delete my.db first=Paul

jsonlitedb query my.db first=Paul

In [14]:
jsonlitedb dump my.db

{"first":"John","last":"Lennon","born":1940,"role":"guitar"}


{"first":"George","last":"Harrison","born":1943,"role":"guitar"}


{"first":"Ringo","last":"Starr","born":1940,"role":"drums"}


{"first":"George","last":"Martin","born":1926,"role":"producer"}


In [15]:
jsonlitedb dump my.db --sql

BEGIN TRANSACTION;


CREATE TABLE items(


    rowid INTEGER PRIMARY KEY AUTOINCREMENT,


    data TEXT


);


...

                    JSON_EXTRACT(data, '$."first"'),JSON_EXTRACT(data, '$."last"')


                );


DELETE FROM "sqlite_sequence";


INSERT INTO "sqlite_sequence" VALUES('items',7);


COMMIT;


In [16]:
jsonlitedb delete my.db --allow-empty
jsonlitedb dump my.db # EMPTY